In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import pickle

from fit_diagnostics import abs_residuals
from plot_utils import plot_res, plot_res_grouped, plot_res_heatmap
from red_likelihood import makeCArray
from red_utils import find_rel_df, split_rel_results

In [ ]:
%matplotlib inline

In [ ]:
jd_time = 2458098.43869
pol = 'ee'
dist = 'cauchy'
dir_path = '.'

In [ ]:
rel_df = find_rel_df(jd_time, pol, dist, dir_path)

In [ ]:
with open('rel_df.{}.{}.md.pkl'.format(jd_time, pol), 'rb') as f:
    md = pickle.load(f)
    
df = pd.read_pickle(rel_df)
df

In [ ]:
# check results for a given frequency & time integration
test_freq = 600
test_tint = 0

resx = df.loc[(test_freq, test_tint)][5:-2].values.astype(float)
test_vis, test_gains = split_rel_results(resx, md['no_unq_bls'])
test_amps = np.abs(test_gains)
test_phases = np.angle(test_gains)
mean_test_amps = np.mean(test_amps)
mean_test_phases = np.mean(test_phases)

print('Gain amplitude solutions for frequency channel {} and time integration {} are:\n{}\n'\
      .format(test_freq, test_tint, test_amps))
print('Gain phase solutions are:\n{}\n'\
      .format(test_phases))
print('Gain amplitude average is: {}'.format(mean_test_amps))
print('Gain phase average is: {}'.format(mean_test_phases))

In [ ]:
# success rate percentage
success_pct = df['success'].sum() / df['success'].size * 100
print('{}% of iterations (freq/tint slice) were succesful'.format(round(success_pct, 2)))

### Number of iterations required

In [ ]:
plot_res(df, 'nit', clip=True)

In [ ]:
plot_res_grouped(df, 'nit', logy=True)

In [ ]:
plot_res_heatmap(df, 'nit', clip=True)

### Log-likelihood

In [ ]:
plot_res(df, 'fun', clip=True)

In [ ]:
plot_res_grouped(df, 'fun', logy=True)

In [ ]:
plot_res_heatmap(df, 'fun', clip=True)

### Residuals

Look at both the real and imaginary of the the median absolute normalized residual

In [ ]:
df[['med_abs_norm_res_Re', 'med_abs_norm_res_Im']] = df.apply(lambda row: \
pd.Series(abs_residuals(row['norm_residual'])), axis=1)

In [ ]:
plot_res_heatmap(df, value='med_abs_norm_res_Re', clip=True)

In [ ]:
plot_res_heatmap(df, value='med_abs_norm_res_Im', clip=True)

### Stability of gains, visibility solutions and degenerate parameters

#### Gains

In [ ]:
no_ants = md['no_ants']
no_resid_p = 4 # number of residual parameters in df dataframe
gains_df = df.iloc[:, -no_ants*2-no_resid_p:-no_resid_p]

gains_df['gamp_mean'] = gains_df.apply(lambda row: \
                        np.mean(np.abs(makeCArray(row[:no_ants*2].values))), \
                        axis=1)
gains_df['gphase_mean'] = gains_df.apply(lambda row: \
                          np.mean(np.angle(makeCArray(row[:no_ants*2].values))), \
                          axis=1)

In [ ]:
plot_res(gains_df, 'gamp_mean', ylim=(0, 2))

In [ ]:
plot_res(gains_df, 'gphase_mean', ylim=(-np.pi, np.pi))

#### Visibility solutions

In [ ]:
no_unq_bls = md['no_unq_bls']
no_min_p = 5 # number of columns in df that are attributes of the SciPy OptimizeResult 
vis_df = df.iloc[:, no_min_p:no_unq_bls*2+no_min_p]

vis_df['vamp_mean'] = vis_df.apply(lambda row: \
                      np.mean(np.abs(makeCArray(row[:no_unq_bls*2].values))), \
                      axis=1)
vis_df['vphase_mean'] = vis_df.apply(lambda row: \
                        np.mean(np.angle(makeCArray(row[:no_unq_bls*2].values))), \
                        axis=1)

In [ ]:
plot_res(vis_df, 'vamp_mean', clip=True, clip_pctile=98)

In [ ]:
plot_res(vis_df, 'vphase_mean', clip=False, ylim=(-np.pi, np.pi))